In [8]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import os

In [9]:
print(os.getcwd())

c:\Users\nhcho\OneDrive\바탕 화면\Github\HotelAI-\notebooks


In [11]:
# 1. 데이터 로드
hotel_df = pd.read_csv("../data/raw/hotel_data.csv")
leisure_df = pd.read_csv("../data/raw/leisure_data.csv")
price_df = pd.read_csv("../data/raw/price.csv")

In [12]:
# 2. 지도 초기화 (중심 좌표 [36.5, 127.5], zoom_start=7)
m = folium.Map(location=[36.5, 127.5], zoom_start=10)

# 3. 호텔 데이터 클러스터 (객실 이미지 포함)
hotel_cluster = MarkerCluster(name='호텔 데이터').add_to(m)
for idx, row in hotel_df.iterrows():
    popup_html = f"""
    <b>숙박업이름:</b> {row['숙박업이름']}<br>
    <b>숙박업유형명:</b> {row['숙박유형명']}<br>
    <b>지역명:</b> {row['지역명']}<br>
    <img src="{row['객실이미지URL']}" width="200" alt="객실 이미지">
    """
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=folium.Popup(popup_html, max_width=300),
        icon=folium.Icon(color='blue', icon='home', prefix='fa')
    ).add_to(hotel_cluster)

# 4. 레저 데이터 클러스터
leisure_cluster = MarkerCluster(name='레저 데이터').add_to(m)
for idx, row in leisure_df.iterrows():
    popup_html = f"""
    <b>레저상품명:</b> {row['레저상품명']}<br>
    <b>시설도로명주소:</b> {row['시설도로명주소']}
    """
    folium.Marker(
        location=[row['시설위도'], row['시설경도']],
        popup=folium.Popup(popup_html, max_width=300),
        icon=folium.Icon(color='green', icon='star', prefix='fa')
    ).add_to(leisure_cluster)

# 5. 가격 데이터 처리: 호텔 데이터를 이용하여 지역별 대표 좌표 계산 후 병합
region_coords = hotel_df.groupby('지역명').agg({'위도': 'mean', '경도': 'mean'}).reset_index()
price_coords_df = pd.merge(price_df, region_coords, on='지역명', how='left')
price_coords_df_clean = price_coords_df.dropna(subset=['위도', '경도'])

price_cluster = MarkerCluster(name='가격 데이터').add_to(m)
for idx, row in price_coords_df_clean.iterrows():
    popup_html = f"""
    <b>숙박업유형명:</b> {row['숙박유형명']}<br>
    <b>평균판매금액:</b> {row['평균판매금액']}<br>
    <b>성수기여부:</b> {row['성수기여부']}
    """
    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius=7,
        color='red',
        fill=True,
        fill_color='red',
        popup=folium.Popup(popup_html, max_width=300)
    ).add_to(price_cluster)

# 6. 레이어 컨트롤 추가 (데이터셋별 표시/숨김 가능)
folium.LayerControl().add_to(m)

# 7. 지도 HTML 파일로 저장 (또는 Jupyter Notebook에서는 m 출력)
m.save("cluster_map_with_image.html")